|Variable|Variável     |Definição	                              |Chave                                         |
|--------|-------------|------------------------------------------|----------------------------------------------|
|survival|sobrevivência|Sobrevivência                             |0 = Não, 1 = Sim                              |
|pclass  |pclass	   |Classe de ingresso                        |1 = 1º, 2 = 2º, 3 = 3º                        |
|ex      |sexo         |Sexo                                      |                                              |
|Age     |Idade em anos| -                                        |                                              |
|sibsp   |sibsp        |Nº de irmãos / cônjuges a bordo do Titanic|	                                             |
|parch   |pergaminho   |Nº de pais / filhos a bordo do Titanic    |                                              |
|ticket  |bilhete      |Número do bilhete	                      |                                              |
|fare    |tarifa       |Tarifa de passageiro                      |	                                             |
|cabin   |cabine       |Número da cabine	                      |                                              |
|embarked|embarcou     |Porto de embarcação                       |C = Cherbourg, Q = Queenstown, S = Southampton|

In [30]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [31]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

In [32]:
combine = [train, test]

In [33]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

# pd.crosstab(train['Title'], train['Sex'])

In [34]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 
                                                'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()    

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [35]:
title = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title)
    dataset['Title'] = dataset['Title'].fillna(0)

In [36]:
# Convertendo em binario
for dataset in combine:
    transf = {"male": 0, "female": 1}
    dataset['Sex_bin'] = dataset['Sex'].map(transf).astype(int)

In [37]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    cab = {"S":1, "C":2, "Q":3}
    dataset['Embarked'] = dataset['Embarked'].map(cab).astype(int)

In [38]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [39]:
for dataset in combine:
    idx = dataset['Age'].loc[dataset['Age']<1].index
    age = dataset['Age'].iloc[idx]*100
    dataset['Age'].iloc[idx] = age

    m = dataset['Age'].median()
    dataset['Age'] = dataset['Age'].fillna(m)

In [40]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
    dataset['Age'] = dataset['Age'].astype(int)

In [41]:
for dataset in combine:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
# train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

In [42]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [43]:
for dataset in combine:
    dataset.set_index('PassengerId', inplace=True)

In [44]:
train_num = train.select_dtypes(include=[np.number])

In [45]:
test_num = test.select_dtypes(include=[np.number])

In [46]:
x, y = train_num.drop('Survived', axis=1), train_num.Survived.copy()

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [48]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
dectree_clf = DecisionTreeClassifier(max_depth=5)
grad_clf = GradientBoostingClassifier()
navies_bayes = GaussianNB()

In [53]:
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf),
                                         ('svc', svm_clf), ('dt', dectree_clf),
                                         ('grad', grad_clf), ('navie', navies_bayes)], voting='hard')

In [54]:
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',...
                                        

In [55]:
for clf in (log_clf,rnd_clf,svm_clf, dectree_clf,grad_clf, navies_bayes, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.7488789237668162
RandomForestClassifier 0.7847533632286996
SVC 0.7757847533632287
DecisionTreeClassifier 0.7802690582959642
GradientBoostingClassifier 0.7757847533632287
GaussianNB 0.7623318385650224
VotingClassifier 0.7757847533632287


In [56]:
ptest = voting_clf.predict(test_num)
prev = pd.Series(ptest, index= test.index, name='Survived')
prev.to_csv("modelo_ensemble_3.csv", header=True)    

In [57]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Sex_bin
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,1,1,0,A/5 21171,0,NaN,1,1,0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,2,1,0,PC 17599,3,C85,2,3,1
3,1,3,"Heikkinen, Miss. Laina",female,1,0,0,STON/O2. 3101282,1,NaN,1,2,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,2,1,0,113803,3,C123,1,3,1
5,0,3,"Allen, Mr. William Henry",male,2,0,0,373450,1,NaN,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,1,0,0,211536,1,NaN,1,5,0
888,1,1,"Graham, Miss. Margaret Edith",female,1,0,0,112053,2,B42,1,2,1
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,1,1,2,W./C. 6607,2,NaN,1,2,1
